In [1]:
%load_ext autoreload
%autoreload 2
import os, sys
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from dataset import *
from models import *

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

SyntaxError: non-default argument follows default argument (models.py, line 42)

In [ ]:
# Load and preprocess data:
path = 'stocks/individual_stocks_5yr/'
stocks = os.listdir(path)
# Load file, pick only open, high, low, close, and convert to numpy array:
ts = pd.read_csv(path+stocks[100]).select_dtypes(['float']).values
# Normalize data to range [-1, 1]:
scaler = MinMaxScaler(feature_range=(-1, 1))
ts_norm = scaler.fit_transform(ts)

In [ ]:
# Setup device:
device = torch.device('cuda') if torch.cuda.is_available() else torch.cpu()

In [ ]:
# Get train loader and test loader:
test_size = 0.2
source_len = 90
target_len = 1
batch_size = 32
trainset, testset, trainloader, testloader = get_datasets_loaders(ts_norm, test_size, source_len, target_len, batch_size)

In [30]:
# Hyper-parameters:
input_size = 4
output_size = 4
hidden_size = 256
num_layers = 2
dropout = 0.2

epochs = 30
lr = 1e-3

total_steps = np.ceil(len(trainset) / batch_size)
print_every = 5

In [32]:
# Initialize model:
model = ManyToOne(input_size, output_size, hidden_size, num_layers, dropout)
model.to(device)

optimizer = optim.Adam(model.parameters(), lr=lr)

# Training loop:
for epoch in range(epochs):
    for i, (src, tgt) in enumerate(trainloader):
        optimizer.zero_grad()
        src, tgt = src.transpose(1, 0).to(device), tgt.transpose(1, 0).to(device)
        output = model(src)
        loss = F.mse_loss(output, tgt.squeeze())
        loss.backward()
        optimizer.step()
        
        # Print out training progress (on same line):
        stats = '[%d/%d][%d/%d]\tLoss: %.4f' \
                % (epoch+1, epochs, i, total_steps, loss.item())
        print('\r' + stats, end="")
        sys.stdout.flush()

        # Print out training progress (on different line):
        if (i % total_steps == 0):
            print('\r' + stats)

[1/30][0/29]	Loss: 0.2454
[2/30][0/29]	Loss: 0.00429
[3/30][0/29]	Loss: 0.00264
[3/30][9/29]	Loss: 0.0015

KeyboardInterrupt: 

#####################################################

In [15]:
%load_ext autoreload
%autoreload 2
import os, sys
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dataset import *
from models import *

from sklearn.preprocessing import MinMaxScaler
import mxnet as mx
from mxnet import nd, init, gluon, context
from mxnet.gluon.data import Dataset, ArrayDataset, DataLoader
from multiprocessing import cpu_count

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
# Load and preprocess data:
path = 'stocks/individual_stocks_5yr/'
stocks = os.listdir(path)
# Load file, pick only open, high, low, close, and convert to numpy array:
ts = pd.read_csv(path+stocks[100]).select_dtypes(['float']).values
# Normalize data to range [-1, 1]:
scaler = MinMaxScaler(feature_range=(-1, 1))
ts_norm = scaler.fit_transform(ts)

In [12]:
# Get train loader and test loader:
test_size = 0.2
source_len = 90
target_len = 1
batch_size = 8
trainset, testset, trainloader, testloader = get_datasets_loaders(ts_norm, test_size, source_len, target_len, batch_size)

In [13]:
# Hyper-parameters:
batch_size = 32
output_size = 4
hidden_size = 256
num_layers = 1
dropout = 0
batch_first = True

epochs = 30
lr = 1e-3

total_steps = np.ceil(len(trainset) / batch_size)
print_every = 5

In [14]:
# Initialize model:
model = ManyToOne(output_size, hidden_size, num_layers, dropout)
model.initialize(init=init.Xavier())
# Choose a loss function:
loss = gluon.loss.L2Loss()
# Choose a trainer:
trainer = gluon.Trainer(model.collect_params(), 'adam', {'learning_rate': 1e-2})

In [19]:
devices = [context.gpu(i) for i in range(context.num_gpus())]

In [22]:
cac, lon = next(iter(trainloader))
cac_list = gluon.utils.split_and_load(cac, devices)

In [24]:
len(cac_list)

1

In [ ]:
# Training loop:
for epoch in range(epochs):
    for i, (src, tgt) in enumerate(trainloader):
        src, tgt = src.as_in_context(mx.gpu(0)), tgt.as_in_context(mx.gpu(0))
        # Forward + backward:
        with autograd.record():
            src, tgt = ctx.to(device), tgt.to(device)
        output = model(ctx)
        loss = F.mse_loss(output, tgt.squeeze())
        loss.backward()
        optimizer.step()
        
        # Print out training progress (on same line):
        stats = '[%d/%d][%d/%d]\tLoss: %.4f' \
                % (epoch+1, epochs, i, total_steps, loss.item())
        print('\r' + stats, end="")
        sys.stdout.flush()

        # Print out training progress (on different line):
        if (i % total_steps == 0):
            print('\r' + stats)